# **RNN**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [10]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    self.hidden_size = hidden_size
    self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
    out, _ = self.rnn(x, h0)
    out = self.fc(out[:, -1, :])
    return out

In [11]:
def VeriOlusturma(seq_length, num_samples):
  X = []
  Y = []
  for _ in range(num_samples):
    x = np.sin(np.linspace(0, 4 * np.pi, seq_length))
    y = np.cos(np.linspace(0, 4 * np.pi, seq_length))
    X.append(x)
    Y.append(y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.float32)

In [12]:
seq_length = 10
num_samples = 100
input_size = 1
hidden_size = 50
output_size = 1

X, Y = VeriOlusturma(seq_length, num_samples)
X = X.unsqueeze(-1)
Y = Y.unsqueeze(-1)

In [13]:
model = RNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [15]:
num_epochs = 100
for epoch in range(num_epochs):
  model.train()
  outputs = model(X)
  optimizer.zero_grad()
  loss = criterion(outputs, Y[:, -1, :])
  loss.backward()
  optimizer.step()

  if (epoch + 1) % 20 == 0:
    print(f'Epoch: [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch: [20/100], Loss: 0.0006
Epoch: [40/100], Loss: 0.0093
Epoch: [60/100], Loss: 0.0015
Epoch: [80/100], Loss: 0.0001
Epoch: [100/100], Loss: 0.0000


# **LSTM & GRU**

In [17]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [18]:
class GRU:
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [19]:
model = LSTM(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
num_epochs = 100
for epoch in range(num_epochs):
  model.train()
  outputs = model(X)
  optimizer.zero_grad()
  loss = criterion(outputs, Y[:, -1, :])
  loss.backward()
  optimizer.step()

  if (epoch + 1) % 20 == 0:
    print(f'Epoch: [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch: [20/100], Loss: 0.2841
Epoch: [40/100], Loss: 0.0001
Epoch: [60/100], Loss: 0.0023
Epoch: [80/100], Loss: 0.0002
Epoch: [100/100], Loss: 0.0001


# **Metin Üretimi**

In [33]:
import string

def yazi_sil(yazi):
  yazi = yazi.lower()
  yazi = yazi.translate(str.maketrans('', '', string.punctuation))
  return yazi

def VeriHazirlama(yazi, seq_length):
  karakter = sorted(list(set(yazi)))
  karakter_to_index = {char: index for index, char in enumerate(karakter)}
  index_to_karakter = {index: char for index, char in enumerate(karakter)}

  X = []
  Y = []
  for i in range(0, len(yazi) - seq_length, 1):
    seq_in = yazi[i:i + seq_length]
    seq_out = yazi[i + seq_length]
    X.append([karakter_to_index[karakter] for karakter in seq_in])
    Y.append(karakter_to_index[seq_out])
  return X, Y, karakter_to_index, index_to_karakter

In [43]:
yazi = "merhaba, bu pytorch'ta rnn kullanan basit bir metin oluşturma örneğidir"
yazi = yazi_sil(yazi)
seq_length = 10
X, Y, karakter_to_index, index_to_karakter = VeriHazirlama(yazi, seq_length)
X = torch.tensor(X, dtype=torch.long)
Y = torch.tensor(Y, dtype=torch.long)

In [47]:
class RNN_yazi(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN_yazi, self).__init__()
    self.hidden_size = hidden_size
    self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x, h):
    out, h = self.rnn(x, h)
    out = self.fc(out[:, -1, :])
    return out, h

  def init_hidden(self, batch_size):
    return torch.zeros(batch_size, self.hidden_size)

In [48]:
model = RNN_yazi(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
num_epochs = 500
batch_size = X.size(0)

for epoch in range(num_epochs):
  model.train()
  hidden = model.init_hidden(batch_size)
  optimizer.zero_grad()
  outputs, hidden = model(X, hidden)
  loss = criterion(outputs, Y)
  loss.backward()
  optimizer.step()

  if (epoch + 1) % 50 == 0:
    print(f'Epoch: [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
def MetinUretme(model, start_str, char_to_idx, idx_to_char, length=100):
    model.eval()
    input_seq = torch.tensor([[char_to_idx[char] for char in start_str]], dtype=torch.float32)
    hidden = model.init_hidden(1)
    generated_str = start_str

    with torch.no_grad():
        for _ in range(length):
            output, hidden = model(input_seq, hidden)
            _, top_idx = torch.max(output, 1)
            predicted_char = index_to_karakter[top_idx.item()]
            generated_str += predicted_char
            input_seq = torch.cat((input_seq[:, 1:], top_idx.unsqueeze(0).unsqueeze(0).float()), dim=1)

    return generated_str

start_str = "hello! World!"
generated_text = MetinUretme(model, start_str, karakter_to_index, index_to_karakter)
print(f'Generated Text: {generated_text}')